In [ ]:
import os
import pandas as pd
from datetime import datetime

# Define file paths
nepse_csv = r"D:\Screener\nepse.csv"  # Replace with the actual path of nepse.csv
nepse_data_folder = r"D:\Screener\nepseData"  # Replace with the actual path of the nepse data folder..

# Read the existing historical nepse data
df = pd.read_csv(nepse_csv)

# Ensure 'Business Date' is in datetime format
df['Business Date'] = pd.to_datetime(df['Business Date'], errors='coerce').dt.date

# Remove any existing duplicates based on 'Business Date' and 'Symbol' in the original data
df = df.drop_duplicates(subset=['Business Date', 'Symbol'], keep='first')

# Iterate through all CSV files in the 'nepse_data' folder
for file_name in os.listdir(nepse_data_folder):
    if file_name.endswith('.csv'):  
        file_path = os.path.join(nepse_data_folder, file_name)

        # Read the new CSV file, skipping bad lines
        new_df = pd.read_csv(file_path, on_bad_lines='skip')

        # Convert the new data columns to the appropriate format
        new_df['Open Price'] = pd.to_numeric(new_df['Open Price'], errors='coerce')
        new_df['High Price'] = pd.to_numeric(new_df['High Price'], errors='coerce')
        new_df['Low Price'] = pd.to_numeric(new_df['Low Price'], errors='coerce')
        new_df['Close Price'] = pd.to_numeric(new_df['Close Price'], errors='coerce')
        new_df['Business Date'] = pd.to_datetime(new_df['Business Date'], errors='coerce').dt.date

        # Remove any existing duplicates in new_df based on 'Business Date' and 'Symbol'
        new_df = new_df.drop_duplicates(subset=['Business Date', 'Symbol'], keep='first')

        # Combine the historical and new data
        df = pd.concat([df, new_df], ignore_index=True)

# Remove duplicates based on 'Business Date' and 'Symbol' after combining all data
df = df.drop_duplicates(subset=['Business Date', 'Symbol'], keep='first')

# Save the final df to nepse.csv
df.to_csv(nepse_csv, index=False)


C:\Users\user\AppData\Local\Temp\ipykernel_8232\1760865596.py:10: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(nepse_csv)


In [2]:
df['Date'] = pd.to_datetime(df['Last Updated Time'], errors='coerce').dt.date
columns_to_drop = ['Last Updated Time', 'Unnamed: 19', 'Last Updated Date','Average Traded Price', 'Total Trades', 'Last Updated Price', 'Security Id', 'Id','Total Traded Value','Previous Day Close Price']

df = df.drop(columns=columns_to_drop, errors='ignore')
df = df.rename(columns={'Total Traded Quantity': 'Volume'})

In [3]:
df['Business Date'] = df['Business Date'].fillna(df['Date'])
df['Business Date'] = pd.to_datetime(df['Business Date'], errors='coerce').dt.date
df = df.dropna(subset=['Business Date'])
df = df.sort_values(by='Business Date', ascending=True)



In [4]:
#EMA
df['200 Day EMA'] = df.groupby('Symbol')['Close Price'].transform(lambda x: x.ewm(span=200, adjust=False).mean())
df['50 Day EMA'] = df.groupby('Symbol')['Close Price'].transform(lambda x: x.ewm(span=50, adjust=False).mean())

In [5]:
pd.set_option('display.max_columns',None)

In [6]:
df[
    (df['Business Date'] == datetime(2025, 2, 17).date()) &
    (df['50 Day EMA']>df['200 Day EMA'])
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA
79065,2025-02-17,NICD88,NIC ASIA Rinpatra 2088,1050.0,1050.0,1050.0,1050.0,50.0,1083.0,935.0,1652.57,2025-02-17,1005.169472,1034.135873
79064,2025-02-17,SDBD87,Shangri-la Development Bank Debenture 2087,1060.0,1060.0,1060.0,1060.0,75.0,1114.0,885.2,795.00,2025-02-17,990.603423,1055.681630
79063,2025-02-17,SRLI,Sanima Reliance Life Insurance Limited,432.6,435.0,425.0,435.0,127910.0,654.0,412.1,21545.83,2025-02-17,422.162756,430.246258
79061,2025-02-17,NMBD87/88,NMB Debenture 8.50% - 2087/88,1031.0,1045.0,1026.2,1045.0,136.0,1082.2,895.0,2090.00,2025-02-17,955.649385,1027.100816
79060,2025-02-17,JBLB,Jeevan Bikas Laghubitta Bittya Sanstha Ltd,1425.0,1430.0,1413.0,1421.0,7720.0,1920.0,1205.0,21829.00,2025-02-17,1316.615048,1373.587892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78957,2025-02-17,BARUN,Barun Hydropower Co. Ltd.,488.0,493.5,476.5,478.5,113458.0,524.0,211.0,2563.87,2025-02-17,371.153123,445.539551
78955,2025-02-17,RIDI,Ridi Power Company Limited,236.0,236.5,231.0,235.5,121973.0,311.0,154.5,5471.49,2025-02-17,226.392502,233.358149
78954,2025-02-17,SHPC,Sanima Mai Hydropower Ltd.,499.0,515.0,486.2,506.8,234890.0,680.0,316.0,18944.15,2025-02-17,438.129152,489.186598
78961,2025-02-17,MERO,Mero Microfinance Bittiya Sanstha Ltd.,768.9,780.3,741.0,778.4,38197.0,980.0,512.0,11045.50,2025-02-17,675.852348,698.183657


In [7]:
#MACD

# Function to calculate MACD Crossover (Bullish or Bearish)
def calculate_macd_crossover(series, short_window=12, long_window=26, signal_window=9):
    # Calculate the short-term (fast) EMA
    fast_ema = series.ewm(span=short_window, min_periods=1).mean()
    
    # Calculate the long-term (slow) EMA
    slow_ema = series.ewm(span=long_window, min_periods=1).mean()
    
    # Calculate MACD as the difference between the fast and slow EMAs
    macd = fast_ema - slow_ema
    
    # Calculate the Signal Line as the EMA of the MACD
    signal_line = macd.ewm(span=signal_window, min_periods=1).mean()
    
    # Assign 'Bullish' or 'Bearish' based on MACD and Signal Line comparison
    crossover = pd.Series('Bearish', index=macd.index)  #'Bearish'
    crossover[macd > signal_line] = 'Bullish'  # 'Bullish' where MACD is above Signal Line
    
    return crossover

# Apply MACD crossover calculation for each stock symbol
df['MACD'] = df.groupby('Symbol')['Close Price'].apply(calculate_macd_crossover).reset_index(level=0, drop=True)


In [8]:
df[
    (df['Business Date'] == datetime(2025, 2, 17).date()) &
    (df['MACD']=='Bullish')
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD
79065,2025-02-17,NICD88,NIC ASIA Rinpatra 2088,1050.00,1050.00,1050.00,1050.00,50.0,1083.00,935.00,1652.57,2025-02-17,1005.169472,1034.135873,Bullish
79064,2025-02-17,SDBD87,Shangri-la Development Bank Debenture 2087,1060.00,1060.00,1060.00,1060.00,75.0,1114.00,885.20,795.00,2025-02-17,990.603423,1055.681630,Bullish
79063,2025-02-17,SRLI,Sanima Reliance Life Insurance Limited,432.60,435.00,425.00,435.00,127910.0,654.00,412.10,21545.83,2025-02-17,422.162756,430.246258,Bullish
79060,2025-02-17,JBLB,Jeevan Bikas Laghubitta Bittya Sanstha Ltd,1425.00,1430.00,1413.00,1421.00,7720.0,1920.00,1205.00,21829.00,2025-02-17,1316.615048,1373.587892,Bullish
79055,2025-02-17,KEF,Kumari Equity Fund,8.76,9.02,8.76,9.02,15200.0,10.76,7.65,902.00,2025-02-17,8.889430,8.987902,Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78959,2025-02-17,NMBMF,NMB Microfinance Bittiya Sanstha Ltd.,820.00,876.00,816.10,864.90,46911.0,918.00,480.00,6239.81,2025-02-17,700.879898,738.098154,Bullish
78958,2025-02-17,JSLBB,Janautthan Samudayic Laghubitta Bittya Sanstha...,1739.10,1840.00,1710.10,1824.00,27143.0,1840.00,839.00,3102.48,2025-02-17,1464.216225,1634.952102,Bullish
78956,2025-02-17,RBCLPO,Rastriya Beema Company Limited Promoter Share,11340.00,11344.00,11160.00,11181.00,390.0,16270.00,9893.10,19860.77,2025-02-17,11864.396210,11658.243009,Bullish
78961,2025-02-17,MERO,Mero Microfinance Bittiya Sanstha Ltd.,768.90,780.30,741.00,778.40,38197.0,980.00,512.00,11045.50,2025-02-17,675.852348,698.183657,Bullish


In [9]:
#RSI
# Function to calculate RSI
def calculate_rsi(series, period=14):
    delta = series.diff()  # Calculate price changes
    gain = (delta.where(delta > 0, 0)).rolling(window=period, min_periods=1).mean()  # Average gains
    loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=1).mean()  # Average losses
    rs = gain / loss  # Relative strength
    rsi = 100 - (100 / (1 + rs))  # RSI formula
    return rsi

# Calculate RSI for each stock symbol
df['RSI'] = df.groupby('Symbol')['Close Price'].apply(calculate_rsi).reset_index(level=0, drop=True)




In [10]:
df[
    (df['Business Date'] == datetime(2025, 2, 17).date()) &
    (df['RSI'] >= 70)
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD,RSI
79060,2025-02-17,JBLB,Jeevan Bikas Laghubitta Bittya Sanstha Ltd,1425.0,1430.0,1413.0,1421.0,7720.0,1920.0,1205.0,21829.00,2025-02-17,1316.615048,1373.587892,Bullish,73.249139
79052,2025-02-17,CYCL,CYC Nepal Laghubitta Bittiya Sanstha Limited,1780.0,1861.5,1744.4,1840.0,23847.0,1923.9,1080.1,4902.21,2025-02-17,1539.061306,1650.258703,Bullish,75.013032
79058,2025-02-17,CHDC,CEDB Holdings Limited,2268.0,2268.0,2180.0,2194.0,124134.0,2370.0,627.0,22691.16,2025-02-17,1500.279052,1825.993946,Bullish,75.827280
79031,2025-02-17,NRN,NRN Infrastructure and Development Limited,1411.0,1445.0,1410.0,1440.0,184331.0,1448.4,478.3,18494.96,2025-02-17,913.095970,1185.520744,Bullish,78.061224
79018,2025-02-17,NICLBSL,NIC ASIA Laghubitta Bittiya Sanstha Limited,696.1,744.0,696.1,734.8,47661.0,907.0,520.3,12781.41,2025-02-17,691.573745,688.377144,Bullish,81.587561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78960,2025-02-17,GILB,Global IME Laghubitta Bittiya Sanstha Ltd.,1399.0,1435.6,1389.1,1418.0,24034.0,1815.0,750.0,8776.00,2025-02-17,1220.252470,1266.207649,Bullish,73.758357
78959,2025-02-17,NMBMF,NMB Microfinance Bittiya Sanstha Ltd.,820.0,876.0,816.1,864.9,46911.0,918.0,480.0,6239.81,2025-02-17,700.879898,738.098154,Bullish,80.731802
78958,2025-02-17,JSLBB,Janautthan Samudayic Laghubitta Bittya Sanstha...,1739.1,1840.0,1710.1,1824.0,27143.0,1840.0,839.0,3102.48,2025-02-17,1464.216225,1634.952102,Bullish,70.028818
78961,2025-02-17,MERO,Mero Microfinance Bittiya Sanstha Ltd.,768.9,780.3,741.0,778.4,38197.0,980.0,512.0,11045.50,2025-02-17,675.852348,698.183657,Bullish,78.034372


In [ ]:
"""simple way to check the trend by just checking if the curennt 15 day high is higher than prev 15 days high for small trend 
   can figure out longer term by checking if 15 days high or todays closing price is higher than prev 60,120,200 high """
#rolling highs 
df['15 Days High'] = df.groupby('Symbol')['High Price'].rolling(window=15).max().reset_index(0, drop=True)
df['30 Days High'] = df.groupby('Symbol')['High Price'].rolling(window=30).max().reset_index(0, drop=True)
df['60 Days High'] = df.groupby('Symbol')['High Price'].rolling(window=60).max().reset_index(0, drop=True)
df['120 Days High'] = df.groupby('Symbol')['High Price'].rolling(window=120).max().reset_index(0, drop=True)
df['200 Days High'] = df.groupby('Symbol')['High Price'].rolling(window=200).max().reset_index(0, drop=True)


#  rolling lows 
df['15 Days Low'] = df.groupby('Symbol')['Low Price'].rolling(window=15).min().reset_index(0, drop=True)
df['30 Days Low'] = df.groupby('Symbol')['Low Price'].rolling(window=30).min().reset_index(0, drop=True)
df['60 Days Low'] = df.groupby('Symbol')['Low Price'].rolling(window=60).min().reset_index(0, drop=True)
df['120 Days Low'] = df.groupby('Symbol')['Low Price'].rolling(window=120).min().reset_index(0, drop=True)
df['200 Days Low'] = df.groupby('Symbol')['Low Price'].rolling(window=200).min().reset_index(0, drop=True)

# track previous values of highs and lows to figure out the trend 
df['prev_15 Days High'] = df.groupby('Symbol')['15 Days High'].shift(15)
df['prev_30 Days High'] = df.groupby('Symbol')['30 Days High'].shift(15)
df['prev_60 Days High'] = df.groupby('Symbol')['60 Days High'].shift(15)
df['prev_120 Days High'] = df.groupby('Symbol')['120 Days High'].shift(15)
df['prev_200 Days High'] = df.groupby('Symbol')['200 Days High'].shift(15)

df['prev_15 Days Low'] = df.groupby('Symbol')['15 Days Low'].shift(15)
df['prev_30 Days Low'] = df.groupby('Symbol')['30 Days Low'].shift(15)
df['prev_60 Days Low'] = df.groupby('Symbol')['60 Days Low'].shift(15)
df['prev_120 Days Low'] = df.groupby('Symbol')['120 Days Low'].shift(15)
df['prev_200 Days Low'] = df.groupby('Symbol')['200 Days Low'].shift(15)


In [411]:
df[
    (df['Business Date']== datetime(2025,2,17).date()) &
    (df['15 Days High'] > df['prev_30 Days High'])&
    (df['Close Price'] > df['prev_60 Days High'])

]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD,RSI,15 Days High,30 Days High,60 Days High,120 Days High,200 Days High,15 Days Low,30 Days Low,60 Days Low,120 Days Low,200 Days Low,prev_15 Days High,prev_30 Days High,prev_60 Days High,prev_120 Days High,prev_200 Days High,prev_15 Days Low,prev_30 Days Low,prev_60 Days Low,prev_120 Days Low,prev_200 Days Low
79058,2025-02-17,CHDC,CEDB Holdings Limited,2268.00,2268.00,2180.00,2194.00,124134.0,2370.00,627.00,22691.16,2025-02-17,1500.279052,1825.993946,Bullish,75.827280,2282.70,2282.70,2282.70,2282.7000,2282.7000,1761.00,1550.200,1545.3000,1504.100,596.3000,1880.00,1880.0000,1880.00000,2109.3000,2109.3000,1550.200,1545.3000,1545.300,1019.05000,596.3000
79084,2025-02-17,HHL,Himalayan Hydropower Limited,585.00,625.00,585.00,597.00,46312.0,635.00,318.00,6537.15,2025-02-17,465.388276,552.474630,Bearish,62.240356,635.00,635.00,635.00,635.0000,635.0000,562.00,492.800,456.4000,389.000,330.5000,592.60,592.6000,592.60000,592.6000,592.6000,492.800,456.4000,432.000,370.00000,318.0000
79068,2025-02-17,NYADI,Nyadi Hydropower Limited,689.10,714.00,666.00,700.00,102609.0,880.00,260.30,10500.00,2025-02-17,421.831228,594.823596,Bearish,58.612942,880.00,880.00,880.00,880.0000,880.0000,585.00,443.000,324.0000,283.300,278.4000,693.00,693.0000,693.00000,693.0000,693.0000,443.000,331.1000,297.100,283.30000,277.6000
79080,2025-02-17,RHGCL,Rapti Hydro And General Construction Limited,674.30,681.00,665.00,669.50,40241.0,700.70,275.00,4102.65,2025-02-17,459.933923,594.896295,Bearish,62.289562,700.70,700.70,700.70,700.7000,700.7000,604.00,533.300,440.0000,354.000,292.0000,648.50,648.5000,648.50000,648.5000,648.5000,533.300,471.2000,376.000,346.10000,286.0000
79067,2025-02-17,SPC,Samling Power Company Limited,667.60,667.60,639.00,642.00,132283.0,722.00,464.00,3210.00,2025-02-17,563.890512,592.177243,Bearish,61.427567,722.00,722.00,722.00,722.0000,722.0000,553.00,485.000,480.0000,471.700,466.0000,600.00,600.0000,600.00000,722.0000,722.0000,485.000,480.0000,471.700,471.70000,466.0000
79031,2025-02-17,NRN,NRN Infrastructure and Development Limited,1411.00,1445.00,1410.00,1440.00,184331.0,1448.40,478.30,18494.96,2025-02-17,913.095970,1185.520744,Bullish,78.061224,1448.40,1448.40,1448.40,1448.4000,1448.4000,1190.00,1025.000,862.6000,834.100,454.3850,1231.90,1231.9000,1231.90000,1231.9000,1231.9000,1025.000,930.0000,840.750,693.59500,454.3850
79028,2025-02-17,NRIC,Nepal Reinsurance Company Limited,961.00,1015.00,961.00,1011.00,325068.0,1052.00,630.10,135698.31,2025-02-17,790.022185,871.972361,Bearish,70.710383,1015.00,1015.00,1015.00,1052.0000,1052.0000,899.00,727.000,722.0000,722.000,630.1000,938.40,938.4000,938.40000,1052.0000,1052.0000,727.000,722.0000,722.000,686.10000,630.1000
79045,2025-02-17,TPC,Terhathum Power Company Limited,620.00,627.00,614.00,618.00,50810.0,671.60,410.00,2472.00,2025-02-17,534.409652,574.982607,Bearish,62.672811,671.60,671.60,671.60,671.6000,671.6000,562.00,512.600,508.0000,410.000,410.0000,601.00,601.0000,601.00000,615.0000,615.0000,512.600,508.0000,446.000,410.00000,410.0000
79035,2025-02-17,UMRH,United IDI Mardi RB Hydropower Limited.,762.00,790.00,742.00,763.20,229413.0,790.00,325.00,3381.74,2025-02-17,536.266323,662.751738,Bullish,64.589235,790.00,790.00,790.00,790.0000,790.0000,682.00,605.000,515.0000,406.700,326.3000,698.00,698.0000,699.80000,699.8000,699.8000,605.000,587.5000,460.000,406.70000,326.3000
79085,2025-02-17,SGHC,Swet-Ganga Hydropower & Construction Limited,629.00,630.10,612.50,612.50,29820.0,649.50,295.20,9757.13,2025-02-17,438.733076,543.068210,Bearish,60.676873,649.50,649.50,649.50,649.5000,649.5000,561.10,443.500,428.0000,351.900,302.1000,585.60,585.6000,585.60000,585.6000,585.6000,443.500,433.0000,378.200,351.90000,300.5000


In [412]:
df[
    (df['Business Date']== datetime(2025,2,17).date()) &
    (df['MACD'] == 'Bullish')&
    (df['Close Price'] > df['50 Day EMA'])&
    (df['50 Day EMA']>df['200 Day EMA'])&
    (df['Close Price'] > df['prev_200 Days High'])
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD,RSI,15 Days High,30 Days High,60 Days High,120 Days High,200 Days High,15 Days Low,30 Days Low,60 Days Low,120 Days Low,200 Days Low,prev_15 Days High,prev_30 Days High,prev_60 Days High,prev_120 Days High,prev_200 Days High,prev_15 Days Low,prev_30 Days Low,prev_60 Days Low,prev_120 Days Low,prev_200 Days Low
79058,2025-02-17,CHDC,CEDB Holdings Limited,2268.00,2268.00,2180.00,2194.00,124134.0,2370.00,627.00,22691.16,2025-02-17,1500.279052,1825.993946,Bullish,75.827280,2282.70,2282.70,2282.70,2282.70,2282.70,1761.00,1550.20,1545.30,1504.100,596.3000,1880.00,1880.00,1880.0,2109.300,2109.300,1550.2,1545.3,1545.30,1019.050,596.300
79031,2025-02-17,NRN,NRN Infrastructure and Development Limited,1411.00,1445.00,1410.00,1440.00,184331.0,1448.40,478.30,18494.96,2025-02-17,913.095970,1185.520744,Bullish,78.061224,1448.40,1448.40,1448.40,1448.40,1448.40,1190.00,1025.00,862.60,834.100,454.3850,1231.90,1231.90,1231.9,1231.900,1231.900,1025.0,930.0,840.75,693.595,454.385
79035,2025-02-17,UMRH,United IDI Mardi RB Hydropower Limited.,762.00,790.00,742.00,763.20,229413.0,790.00,325.00,3381.74,2025-02-17,536.266323,662.751738,Bullish,64.589235,790.00,790.00,790.00,790.00,790.00,682.00,605.00,515.00,406.700,326.3000,698.00,698.00,699.8,699.800,699.800,605.0,587.5,460.00,406.700,326.300
79129,2025-02-17,MEHL,Manakamana Engineering Hydropower Limited,656.00,679.00,656.00,670.50,73396.0,679.00,275.00,5364.00,2025-02-17,424.559530,530.648171,Bullish,83.453637,679.00,679.00,679.00,679.00,679.00,542.00,443.10,411.60,344.200,290.0000,577.20,577.20,577.2,577.200,577.200,443.1,432.1,363.60,341.300,283.500
79143,2025-02-17,ULHC,Upper Lohore Khola Hydropower Company Limited,632.00,638.00,605.10,620.10,54755.0,693.00,300.00,3161.29,2025-02-17,449.612385,547.948383,Bullish,62.017080,693.00,693.00,693.00,693.00,693.00,545.00,476.50,432.70,364.000,317.1000,586.50,586.50,586.5,586.500,586.500,476.5,455.0,384.50,364.000,306.100
79086,2025-02-17,GIBF1,Global IME Balanced Fund-1,9.69,10.61,9.55,10.61,3500.0,10.61,7.92,1088.34,2025-02-17,9.454903,9.878453,Bullish,64.000000,10.61,10.61,10.61,10.61,10.61,9.46,9.46,9.46,8.840,7.9200,10.15,10.43,10.5,10.600,10.600,9.5,9.5,9.11,8.590,7.920
79099,2025-02-17,MHCL,Molung Hydropower Company Limited,640.50,640.50,620.00,624.90,88825.0,640.50,355.00,5014.82,2025-02-17,482.070363,554.111682,Bullish,72.360845,640.50,640.50,640.50,640.50,640.50,534.00,511.00,421.00,355.100,355.1000,574.00,592.00,592.0,592.000,592.000,511.0,481.0,392.00,355.100,355.100
79088,2025-02-17,SIKLES,Sikles Hydropower Limited,1355.00,1475.00,1341.00,1387.90,5471.0,1475.00,452.00,11797.15,2025-02-17,792.633301,1035.435547,Bullish,76.491733,1475.00,1475.00,1475.00,1475.00,1475.00,1026.60,803.10,738.20,637.000,467.0000,1006.50,1006.50,1006.5,1006.500,1006.500,803.1,769.2,685.00,637.000,455.000
79120,2025-02-17,MAKAR,Makar Jitumaya Suri Hydropower Limited,827.00,835.00,803.60,813.00,72796.0,860.00,342.00,6178.80,2025-02-17,565.964227,709.385030,Bullish,71.215881,860.00,860.00,860.00,860.00,860.00,687.00,583.00,580.00,431.300,342.0000,706.00,706.00,770.0,770.000,770.000,583.0,580.0,464.00,431.300,342.000
79115,2025-02-17,PHCL,Peoples Hydropower Company Limited,687.20,687.20,668.00,680.00,225228.0,701.20,275.10,21760.00,2025-02-17,466.699128,578.746195,Bullish,76.237624,701.20,701.20,701.20,701.20,701.20,563.60,517.00,491.00,310.000,275.1000,618.00,618.00,670.0,670.000,670.000,517.0,491.0,409.00,310.000,275.100


In [413]:
pd.set_option('display.max_columns',None)

In [ ]:
#KEY LEVEL 
""" finds out the nearest key level by calculating key price levels for each stock symbol by
    taking the most frequent high price (mode) over different time frames (200, 120, 60, and 30 days), 
    then assign the key level as the closest mode price to the current closing price """

time_frames = [200, 120, 60, 30]

for symbol in df["Symbol"].unique():
    stock_data = df[df["Symbol"] == symbol].sort_values("Date", ascending=False)

    key_levels = []

    for days in time_frames:
        selected_data = stock_data.head(days)["High Price"]

        mode_price = selected_data.mode().values

        if len(mode_price) > 0:
            key_levels.append(mode_price[0])

    key_levels = sorted(set(key_levels))

    current_price = stock_data.iloc[0]["Close Price"]
    key_level = min(key_levels, key=lambda x: abs(x - current_price)) if key_levels else None

    df.loc[df["Symbol"] == symbol, "Key Level"] = key_level

In [ ]:
#KEY ZONE 
""" by making the range 1% below and above key price so that can check if the closing price is in key zone """

'by making the range 1% below and above key price so that can check if the closing price is in key zone '

In [428]:
"""def calculate_nearest_support_resistance(df, column_high='High Price', column_low='Low Price', windows=[30, 60, 120, 200]):
    df = df.sort_values(by=['Business Date']).reset_index(drop=True)

    df['Support'] = np.nan
    df['Resistance'] = np.nan

    for symbol, symbol_data in df.groupby('Symbol'):
        data_length = len(symbol_data)
        relevant_windows = [w for w in windows if w <= data_length]
        
        if not relevant_windows:
            continue

        for i in range(max(relevant_windows) - 1, len(symbol_data)):
            current_price = symbol_data.iloc[i]['Close Price']
            
            nearest_support = None
            nearest_resistance = None

            for window in relevant_windows:
                lookback_data_high = symbol_data.iloc[i - window + 1:i + 1][column_high]
                lookback_data_low = symbol_data.iloc[i - window + 1:i + 1][column_low]
                
                window_support = lookback_data_low.min() 
                window_resistance = lookback_data_high.max()
                
                if nearest_support is None or (window_support <= current_price and abs(window_support - current_price) < abs(nearest_support - current_price)):
                    nearest_support = window_support

                if nearest_resistance is None or (window_resistance >= current_price and abs(window_resistance - current_price) < abs(nearest_resistance - current_price)):
                    nearest_resistance = window_resistance

       
            if current_price < nearest_support:  
                for next_window in relevant_windows:  
                    if next_window > windows[relevant_windows.index(min(relevant_windows))]:  
                        lookback_data_low = symbol_data.iloc[i - next_window + 1:i + 1][column_low]
                        nearest_support = lookback_data_low.min() 
                        break  

            if current_price > nearest_resistance:
                for next_window in relevant_windows: 
                    if next_window > windows[relevant_windows.index(min(relevant_windows))]:  
                        lookback_data_high = symbol_data.iloc[i - next_window + 1:i + 1][column_high]
                        nearest_resistance = lookback_data_high.max()  
                        break 

            df.loc[symbol_data.index[i], 'Support'] = nearest_support
            df.loc[symbol_data.index[i], 'Resistance'] = nearest_resistance

    return df

df = calculate_nearest_support_resistance(df, column_high='High Price', column_low='Low Price', windows=[30, 60, 120, 200])
"""

"def calculate_nearest_support_resistance(df, column_high='High Price', column_low='Low Price', windows=[30, 60, 120, 200]):\n    df = df.sort_values(by=['Business Date']).reset_index(drop=True)\n\n    df['Support'] = np.nan\n    df['Resistance'] = np.nan\n\n    for symbol, symbol_data in df.groupby('Symbol'):\n        data_length = len(symbol_data)\n        relevant_windows = [w for w in windows if w <= data_length]\n        \n        if not relevant_windows:\n            continue\n\n        for i in range(max(relevant_windows) - 1, len(symbol_data)):\n            current_price = symbol_data.iloc[i]['Close Price']\n            \n            nearest_support = None\n            nearest_resistance = None\n\n            for window in relevant_windows:\n                lookback_data_high = symbol_data.iloc[i - window + 1:i + 1][column_high]\n                lookback_data_low = symbol_data.iloc[i - window + 1:i + 1][column_low]\n                \n                window_support = lookback_da

In [417]:
"""import pandas as pd

# Assuming df is your dataframe
df['pivot_high'] = None
df['pivot_low'] = None

# Define window for pivot high and low
lookback = 20  # Look 20 days before and after

# Sort the data by 'Business Date'
df = df.sort_values(by=['Business Date']).reset_index(drop=True)

# Group by 'Symbol' and calculate pivots for each symbol
for symbol, group in df.groupby('Symbol'):
    # Reset the index for each group to ensure we can access by integer position
    group = group.reset_index(drop=True)
    
    # Loop through the rows where there are enough surrounding data
    for i in range(lookback, len(group) - lookback):
        current_high = group.loc[i, 'High Price']
        current_low = group.loc[i, 'Low Price']
        
        # Calculate the range for previous and next 20 days
        prev_20_days_high = group['High Price'][i - lookback:i]
        next_20_days_high = group['High Price'][i + 1:i + lookback + 1]
        
        prev_20_days_low = group['Low Price'][i - lookback:i]
        next_20_days_low = group['Low Price'][i + 1:i + lookback + 1]
        
        
        # Check if current_high is the highest in the range
        if current_high > max(prev_20_days_high.max(), next_20_days_high.max()):
            df.loc[group.index[i], 'pivot_high'] = current_high
            print(f"Pivot High set for index {i}")

        # Check if current_low is the lowest in the range
        if current_low < min(prev_20_days_low.min(), next_20_days_low.min()):
            df.loc[group.index[i], 'pivot_low'] = current_low
            print(f"Pivot Low set for index {i}")

# After the loop, check the result for 'NTC' symbol
print(df[df['Symbol']=='NTC'].tail(10))  # Check the last few rows for NTC
"""

'import pandas as pd\n\n# Assuming df is your dataframe\ndf[\'pivot_high\'] = None\ndf[\'pivot_low\'] = None\n\n# Define window for pivot high and low\nlookback = 20  # Look 20 days before and after\n\n# Sort the data by \'Business Date\'\ndf = df.sort_values(by=[\'Business Date\']).reset_index(drop=True)\n\n# Group by \'Symbol\' and calculate pivots for each symbol\nfor symbol, group in df.groupby(\'Symbol\'):\n    # Reset the index for each group to ensure we can access by integer position\n    group = group.reset_index(drop=True)\n    \n    # Loop through the rows where there are enough surrounding data\n    for i in range(lookback, len(group) - lookback):\n        current_high = group.loc[i, \'High Price\']\n        current_low = group.loc[i, \'Low Price\']\n        \n        # Calculate the range for previous and next 20 days\n        prev_20_days_high = group[\'High Price\'][i - lookback:i]\n        next_20_days_high = group[\'High Price\'][i + 1:i + lookback + 1]\n        \n    

In [418]:
"""import pandas as pd

# Assuming df is your dataframe
df['pivot_high'] = None
df['pivot_low'] = None

# Define window for pivot high and low
lookback = 20  # Look 20 days before and after

# Sort the data by 'Business Date'
df = df.sort_values(by=['Business Date']).reset_index(drop=True)

# Group by 'Symbol' and calculate pivots for each symbol
for symbol, group in df.groupby('Symbol'):
    # Reset the index for each group to ensure we can access by integer position
    group = group.reset_index(drop=True)
    
    # Loop through the rows where there are enough surrounding data
    for i in range(lookback, len(group) - lookback):
        current_high = group.loc[i, 'High Price']
        current_low = group.loc[i, 'Low Price']
        
        # Calculate the range for previous and next 20 days
        prev_20_days_high = group['High Price'][i - lookback:i]
        next_20_days_high = group['High Price'][i + 1:i + lookback + 1]
        
        prev_20_days_low = group['Low Price'][i - lookback:i]
        next_20_days_low = group['Low Price'][i + 1:i + lookback + 1]
        
        # Check if current_high is the highest in the range
        if current_high > max(prev_20_days_high.max(), next_20_days_high.max()):
            df.loc[group.index[i], 'pivot_high'] = current_high

        # Check if current_low is the lowest in the range
        if current_low < min(prev_20_days_low.min(), next_20_days_low.min()):
            df.loc[group.index[i], 'pivot_low'] = current_low

# Forward fill the 'pivot_high' and 'pivot_low' columns
df['pivot_high'] = df['pivot_high'].fillna(method='ffill')
df['pivot_low'] = df['pivot_low'].fillna(method='ffill')

# Display the updated DataFrame for 'NTC' symbol or the entire DataFrame
print(df[df['Symbol'] == 'NTC'].tail(10))  # Check the last few rows for NTC



"""

"import pandas as pd\n\n# Assuming df is your dataframe\ndf['pivot_high'] = None\ndf['pivot_low'] = None\n\n# Define window for pivot high and low\nlookback = 20  # Look 20 days before and after\n\n# Sort the data by 'Business Date'\ndf = df.sort_values(by=['Business Date']).reset_index(drop=True)\n\n# Group by 'Symbol' and calculate pivots for each symbol\nfor symbol, group in df.groupby('Symbol'):\n    # Reset the index for each group to ensure we can access by integer position\n    group = group.reset_index(drop=True)\n    \n    # Loop through the rows where there are enough surrounding data\n    for i in range(lookback, len(group) - lookback):\n        current_high = group.loc[i, 'High Price']\n        current_low = group.loc[i, 'Low Price']\n        \n        # Calculate the range for previous and next 20 days\n        prev_20_days_high = group['High Price'][i - lookback:i]\n        next_20_days_high = group['High Price'][i + 1:i + lookback + 1]\n        \n        prev_20_days_low

In [419]:
"""import pandas as pd

# Assuming df is your dataframe

# Define the window size for the left and right periods
left_window = 20
right_window = 20

# Function to calculate pivot high and pivot low
def calculate_pivot_high_low(group):
    # Calculate Pivot High: The highest high in the window of 20 candles before and 20 candles after
    group['pivot_high'] = group['High Price'].rolling(window=left_window + right_window + 1, 
                                                      min_periods=1, 
                                                      center=True).max()

    # Calculate Pivot Low: The lowest low in the window of 20 candles before and 20 candles after
    group['pivot_low'] = group['Low Price'].rolling(window=left_window + right_window + 1, 
                                                     min_periods=1, 
                                                     center=True).min()

    # Forward fill any missing values (if necessary)
    group['pivot_high'] = group['pivot_high'].ffill()
    group['pivot_low'] = group['pivot_low'].ffill()
    
    return group

# Group by Symbol and apply the calculation for each group (each symbol)
df = df.groupby('Symbol').apply(calculate_pivot_high_low)
"""

"import pandas as pd\n\n# Assuming df is your dataframe\n\n# Define the window size for the left and right periods\nleft_window = 20\nright_window = 20\n\n# Function to calculate pivot high and pivot low\ndef calculate_pivot_high_low(group):\n    # Calculate Pivot High: The highest high in the window of 20 candles before and 20 candles after\n    group['pivot_high'] = group['High Price'].rolling(window=left_window + right_window + 1, \n                                                      min_periods=1, \n                                                      center=True).max()\n\n    # Calculate Pivot Low: The lowest low in the window of 20 candles before and 20 candles after\n    group['pivot_low'] = group['Low Price'].rolling(window=left_window + right_window + 1, \n                                                     min_periods=1, \n                                                     center=True).min()\n\n    # Forward fill any missing values (if necessary)\n    group['pivot_high'

In [420]:
"""import pandas as pd
import numpy as np

def calculate_support_resistance(df, column_high='High Price', column_low='Low Price', column_close='Close Price', column_volume='Volume', lookback_period=20, vol_len=2, box_width=1):
    df = df.sort_values(by=['Business Date']).reset_index(drop=True)
    
    # Create new columns to store the results
    df['Support Level'] = np.nan
    df['Resistance Level'] = np.nan
    df['Support Break'] = False
    df['Resistance Break'] = False
    df['Support Holds'] = False
    df['Resistance Holds'] = False

    # Calculate the Volume Delta (buy/sell volume)
    def up_and_down_volume(df):
        df['Pos Volume'] = np.where(df['Close Price'] > df['Open Price'], df[column_volume], 0)
        df['Neg Volume'] = np.where(df['Close Price'] < df['Open Price'], df[column_volume], 0)
        df['Total Volume'] = df['Pos Volume'] + df['Neg Volume']
        return df

    df = up_and_down_volume(df)

    # Loop through each symbol in the DataFrame (assuming there is a 'Symbol' column)
    for symbol, symbol_data in df.groupby('Symbol'):
        data_length = len(symbol_data)

        # Calculate the support and resistance levels
        for i in range(lookback_period, data_length):
            current_price = symbol_data.iloc[i][column_close]
            high_vol = symbol_data.iloc[i - vol_len:i][column_volume].max() / 2.5
            low_vol = symbol_data.iloc[i - vol_len:i][column_volume].min() / 2.5
            
            # Ensure there is enough data for the lookback period
            if i - lookback_period < 0:
                continue
            
            # Find pivot points
            pivot_high = symbol_data.iloc[i - lookback_period:i][column_high].idxmax()  # Highest high in lookback period
            pivot_low = symbol_data.iloc[i - lookback_period:i][column_low].idxmin()    # Lowest low in lookback period
            
            # Ensure pivots are within bounds
            if pivot_high < 0 or pivot_low < 0 or pivot_high >= data_length or pivot_low >= data_length:
                continue

            vol = symbol_data.iloc[i][column_volume]
            
            # Ensure valid values before comparison
            if pd.notna(vol) and pd.notna(high_vol) and pd.notna(low_vol):
                support_level = symbol_data.iloc[pivot_low][column_low] if vol > high_vol else np.nan
                resistance_level = symbol_data.iloc[pivot_high][column_high] if vol < low_vol else np.nan
            else:
                support_level = np.nan
                resistance_level = np.nan

            # Add support and resistance levels to the DataFrame
            df.loc[symbol_data.index[i], 'Support Level'] = support_level
            df.loc[symbol_data.index[i], 'Resistance Level'] = resistance_level

            # Check if support or resistance levels were broken
            if pd.notna(support_level) and current_price < support_level:
                df.loc[symbol_data.index[i], 'Support Break'] = True
                df.loc[symbol_data.index[i], 'Support Holds'] = False
            elif pd.notna(support_level) and current_price > support_level:
                df.loc[symbol_data.index[i], 'Support Holds'] = True

            if pd.notna(resistance_level) and current_price > resistance_level:
                df.loc[symbol_data.index[i], 'Resistance Break'] = True
                df.loc[symbol_data.index[i], 'Resistance Holds'] = False
            elif pd.notna(resistance_level) and current_price < resistance_level:
                df.loc[symbol_data.index[i], 'Resistance Holds'] = True

    return df

# Example usage:
# Assuming df has the columns ['Symbol', 'Business Date', 'High Price', 'Low Price', 'Close Price', 'Volume']
df = calculate_support_resistance(df, column_high='High Price', column_low='Low Price', column_close='Close Price', column_volume='Volume', lookback_period=20, vol_len=2, box_width=1)
"""

"import pandas as pd\nimport numpy as np\n\ndef calculate_support_resistance(df, column_high='High Price', column_low='Low Price', column_close='Close Price', column_volume='Volume', lookback_period=20, vol_len=2, box_width=1):\n    df = df.sort_values(by=['Business Date']).reset_index(drop=True)\n    \n    # Create new columns to store the results\n    df['Support Level'] = np.nan\n    df['Resistance Level'] = np.nan\n    df['Support Break'] = False\n    df['Resistance Break'] = False\n    df['Support Holds'] = False\n    df['Resistance Holds'] = False\n\n    # Calculate the Volume Delta (buy/sell volume)\n    def up_and_down_volume(df):\n        df['Pos Volume'] = np.where(df['Close Price'] > df['Open Price'], df[column_volume], 0)\n        df['Neg Volume'] = np.where(df['Close Price'] < df['Open Price'], df[column_volume], 0)\n        df['Total Volume'] = df['Pos Volume'] + df['Neg Volume']\n        return df\n\n    df = up_and_down_volume(df)\n\n    # Loop through each symbol in th

In [ ]:
# range & breakout (simple logic for box)
""" checks if the last follwing days in within the range of following % .. 7(5%), 15(10%), 30(10%) 45(15%)
    identifies breakout signals by checking if the closing price exceeds the highest or lowest price in the last 7(5%), 15(10%), 30(10%),45(15%) """

periods = [7, 15, 30, 45]
percentages = {7: 0.05, 15: 0.10, 30: 0.10, 45: 0.15}

df.sort_values(by=['Symbol', 'Business Date'], inplace=True)

for period in periods:
    df[f'Last_{period}_Days_High'] = df.groupby('Symbol')['High Price'].rolling(window=period, min_periods=1).max().shift(0).reset_index(level=0, drop=True)
    df[f'Last_{period}_Days_Low'] = df.groupby('Symbol')['Low Price'].rolling(window=period, min_periods=1).min().shift(0).reset_index(level=0, drop=True)

    df[f'{period}_days_Within_{int(percentages[period] * 100)}_Percent'] = df[f'Last_{period}_Days_High'] <= (1 + percentages[period]) * df[f'Last_{period}_Days_Low']

    df[f'{period}_days_Bullish_Breakout'] = df['Close Price'] > df[f'Last_{period}_Days_High']
    df[f'{period}_days_Bearish_Breakout'] = df['Close Price'] < df[f'Last_{period}_Days_Low']

def breakout_period(row):
    for period in periods:
        if row[f'{period}_days_Bullish_Breakout']:
            return f'Bullish Breakout ({period} days)'
        elif row[f'{period}_days_Bearish_Breakout']:
            return f'Bearish Breakout ({period} days)'
    return 'No Breakout'

df['Breakout_Type'] = df.apply(breakout_period, axis=1)


In [422]:
df[
    (df['Business Date']== datetime(2025,2,17).date()) & 
    (df['7_days_Within_5_Percent'] == True) 
    #(df['7_days_Bullish_Breakout'] == T)&
    #(df['Breakout_Type'] != 'No Breakout')
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD,RSI,15 Days High,30 Days High,60 Days High,120 Days High,200 Days High,15 Days Low,30 Days Low,60 Days Low,120 Days Low,200 Days Low,prev_15 Days High,prev_30 Days High,prev_60 Days High,prev_120 Days High,prev_200 Days High,prev_15 Days Low,prev_30 Days Low,prev_60 Days Low,prev_120 Days Low,prev_200 Days Low,Key Level,Last_7_Days_High,Last_7_Days_Low,7_days_Within_5_Percent,7_days_Bullish_Breakout,7_days_Bearish_Breakout,Last_15_Days_High,Last_15_Days_Low,15_days_Within_10_Percent,15_days_Bullish_Breakout,15_days_Bearish_Breakout,Last_30_Days_High,Last_30_Days_Low,30_days_Within_10_Percent,30_days_Bullish_Breakout,30_days_Bearish_Breakout,Last_45_Days_High,Last_45_Days_Low,45_days_Within_15_Percent,45_days_Bullish_Breakout,45_days_Bearish_Breakout,Breakout_Type
78943,2025-02-17,ADBL,Agricultural Development Bank Limited,292.0,300.9,289.5,295.0,70893.0,428.4,220.5,40872.91,2025-02-17,305.881407,305.894044,Bullish,60.277778,304.9,310.0,360.258,415.548,415.548,281.1,275.1,275.1,275.1000,213.88500,310.0,338.53,383.053,415.548,415.548,275.1,275.1,275.1,275.10000,213.885,295.0,301.0,289.3,True,False,False,304.9,281.1,True,False,False,310.0,275.1,False,False,False,338.53,275.1,False,False,False,No Breakout
79017,2025-02-17,ADBLD83,10.35% Agricultural Bank Debenture 2083,1100.0,1100.0,1100.0,1100.0,250.0,1160.0,1010.0,2750.00,2025-02-17,1082.292997,1106.605549,Bearish,44.390426,1150.0,1160.0,1160.000,1160.000,NaN,1079.9,1079.9,1060.0,1010.0000,NaN,1160.0,1160.00,1160.000,1160.000,NaN,1091.0,1090.0,1052.3,966.00000,NaN,1100.0,1101.4,1079.9,True,False,False,1150.0,1079.9,True,False,False,1160.0,1079.9,True,False,False,1160.00,1079.9,True,False,False,No Breakout
78923,2025-02-17,ALICL,Asian Life Insurance Co. Limited,755.0,775.0,755.0,773.0,31171.0,825.0,523.0,26280.74,2025-02-17,703.111084,742.577656,Bullish,61.606161,795.0,795.0,795.000,825.000,825.000,732.0,693.4,693.4,654.1000,523.00000,761.0,761.00,780.000,825.000,825.000,693.4,693.4,693.4,615.00000,523.000,725.0,780.0,744.8,True,False,False,795.0,732.0,True,False,False,795.0,693.4,False,False,False,795.00,693.4,True,False,False,No Breakout
78878,2025-02-17,BNT,Bottlers Nepal (Terai) Limited,13265.1,13350.0,13016.0,13209.9,215.0,18100.0,11960.1,15983.98,2025-02-17,13868.559831,13715.106922,Bearish,42.174265,14191.0,14600.0,14911.000,17523.000,18100.000,12979.0,12979.0,12750.0,12750.0000,12208.10000,14600.0,14600.00,15289.700,18100.000,18100.000,13200.0,12750.0,12750.0,12546.00000,12208.100,13000.0,13535.5,12979.0,True,False,False,14191.0,12979.0,True,False,False,14600.0,12979.0,False,False,False,14600.00,12750.0,True,False,False,No Breakout
78895,2025-02-17,CBBL,Chhimek Laghubitta Bittiya Sanstha Limited,880.0,891.0,872.0,888.0,24719.0,1155.0,800.0,28552.27,2025-02-17,856.556909,866.220459,Bullish,62.393162,899.0,899.0,913.560,1062.600,1062.600,840.0,835.0,828.0,828.0000,736.00000,870.9,887.80,929.016,1062.600,1062.600,835.0,828.0,828.0,814.20000,736.000,901.6,891.0,860.0,True,False,False,899.0,840.0,True,False,False,899.0,835.0,True,False,False,899.00,828.0,True,False,False,No Breakout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78892,2025-02-17,STC,Salt Trading Corporation,5220.0,5309.0,5220.0,5255.0,2416.0,7000.0,4210.0,16853.68,2025-02-17,5522.242348,5481.301290,Bullish,40.486907,5467.0,5720.0,6399.000,7000.000,7000.000,5150.0,5150.0,5150.0,5150.0000,4210.00000,5720.0,5976.00,6399.000,7000.000,7000.000,5250.0,5250.0,5250.0,4836.00000,4210.000,5250.0,5390.0,5150.0,True,False,False,5467.0,5150.0,True,False,False,5720.0,5150.0,False,False,False,5976.00,5150.0,False,False,False,No Breakout
78915,2025-02-17,SWBBL,Sw

In [ ]:
# 10 days avg candle spread / 20 days avg volumn spread
"""volume and price spread by calculating the average candle spread over 10 days and the average volume over 20 days"""

def volume_spread_analysis(group):
    spread = group['High Price'] - group['Low Price']    
    avg_volume = group['Volume'].rolling(window=20).mean()
    ema_spread = spread.ewm(span=10).mean()

    vsa_bullish = (group['Volume'] > avg_volume) & (spread > ema_spread) & (group['High Price'] > group['High Price'].shift(1))
    vsa_bearish = (group['Volume'] > avg_volume) & (spread < ema_spread) & (group['Low Price'] < group['Low Price'].shift(1))

    group['VSA Bullish'] = vsa_bullish
    group['VSA Bearish'] = vsa_bearish

    return group

df = df.groupby('Symbol', group_keys=False).apply(volume_spread_analysis)




C:\Users\user\AppData\Local\Temp\ipykernel_5664\3750385195.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Symbol', group_keys=False).apply(volume_spread_analysis)


In [424]:
df[
    (df['Business Date']== datetime(2025,2,17).date()) & 
    (df['VSA Bearish']==True)
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD,RSI,15 Days High,30 Days High,60 Days High,120 Days High,200 Days High,15 Days Low,30 Days Low,60 Days Low,120 Days Low,200 Days Low,prev_15 Days High,prev_30 Days High,prev_60 Days High,prev_120 Days High,prev_200 Days High,prev_15 Days Low,prev_30 Days Low,prev_60 Days Low,prev_120 Days Low,prev_200 Days Low,Key Level,Last_7_Days_High,Last_7_Days_Low,7_days_Within_5_Percent,7_days_Bullish_Breakout,7_days_Bearish_Breakout,Last_15_Days_High,Last_15_Days_Low,15_days_Within_10_Percent,15_days_Bullish_Breakout,15_days_Bearish_Breakout,Last_30_Days_High,Last_30_Days_Low,30_days_Within_10_Percent,30_days_Bullish_Breakout,30_days_Bearish_Breakout,Last_45_Days_High,Last_45_Days_Low,45_days_Within_15_Percent,45_days_Bullish_Breakout,45_days_Bearish_Breakout,Breakout_Type,VSA Bullish,VSA Bearish
79120,2025-02-17,MAKAR,Makar Jitumaya Suri Hydropower Limited,827.0,835.0,803.6,813.0,72796.0,860.0,342.0,6178.80,2025-02-17,565.964227,709.385030,Bullish,71.215881,860.0,860.0,860.0,860.0,860.0,687.0,583.0,580.0,431.3,342.0000,706.0,706.0,770.0,770.0,770.0,583.0,580.0,464.0,431.3,342.0000,657.0,859.0,780.2,False,False,False,860.0,687.0,False,False,False,860.0,583.0,False,False,False,860.0,580.0,False,False,False,No Breakout,False,True
78941,2025-02-17,MATRI,Matribhumi Lagubitta Bittiya Sanstha Limited,1430.0,1430.0,1365.0,1386.5,14259.0,1716.0,775.0,8712.18,2025-02-17,1187.567939,1278.239827,Bullish,77.921332,1464.8,1464.8,1464.8,1716.0,1716.0,1170.0,1150.0,1150.0,972.0,775.0000,1290.0,1320.0,1482.0,1716.0,1716.0,1150.0,1150.0,1150.0,963.5,775.0000,1430.0,1464.8,1231.2,False,False,False,1464.8,1170.0,False,False,False,1464.8,1150.0,False,False,False,1464.8,1150.0,False,False,False,No Breakout,False,True
78930,2025-02-17,MPFL,Multipurpose Finance Company Limited,760.2,760.2,723.0,725.0,60841.0,936.9,328.0,4423.95,2025-02-17,628.212070,695.296779,Bullish,72.145329,775.0,775.0,839.4,936.9,936.9,581.2,581.2,581.2,549.0,356.1000,749.0,770.0,839.4,936.9,936.9,615.0,595.0,595.0,520.0,344.1000,729.0,775.0,687.0,False,False,False,775.0,581.2,False,False,False,775.0,581.2,False,False,False,775.0,581.2,False,False,False,No Breakout,False,True
79128,2025-02-17,SNLI,Sun Nepal Life Insurance Company Limited,680.3,683.9,676.0,676.6,58941.0,708.0,447.2,26804.19,2025-02-17,580.702239,640.303593,Bullish,66.813187,701.0,701.0,701.0,708.0,708.0,631.0,607.0,607.0,530.0,440.2836,658.0,664.0,676.3,708.0,708.0,607.0,607.0,574.0,505.1,439.8264,630.0,701.0,638.3,False,False,False,701.0,631.0,False,False,False,701.0,607.0,False,False,False,701.0,607.0,False,False,False,No Breakout,False,True
78872,2025-02-17,SPIL,Siddhartha Premier Insurance Limited,821.0,838.0,820.0,835.9,28211.0,1076.0,706.0,23459.95,2025-02-17,832.988955,839.383768,Bearish,55.934613,899.0,899.0,907.5,1076.0,1076.0,804.0,785.2,778.0,778.0,711.5000,844.0,845.0,915.0,1076.0,1076.0,785.2,778.0,778.0,778.0,711.5000,816.0,888.7,820.0,False,False,False,899.0,804.0,False,False,False,899.0,785.2,False,False,False,899.0,778.0,False,False,False,No Breakout,False,True
79073,2025-02-17,ULBSL,Upakar Laghubitta Bittiya Sanstha Limited,4885.3,4990.0,4620.0,4900.0,7106.0,6600.0,1045.0,5201.26,2025-02-17,3122.291917,4404.823578,Bearish,53.124945,6600.0,6600.0,6600.0,6600.0,6600.0,4107.0,3876.0,2951.0,2650.5,1200.0000,4936.8,4936.8,4936.8,4936.8,4936.8,3876.0,3030.1,2951.0,1970.0,1075.0000,4100.0,6300.0,4620.0,False,False,False,6600.0,4107.0,False,False,False,6600.0,3876.0,False,False,False,6600.0,3030.1,False,False,False,No Breakout,False,True


In [ ]:
#Volumn diivergence

""" detects bullish and dearish divergence based on price and volume 
    bullish divergence-price makes lower lows over 14 days, but volume increases over the past 7 days
    bearish Divergence- price makes higher highs over 14 days, but volume decreases over the past 7 days"""

def volume_divergence(group):
    short_window = 7  
    long_window = 14  

    group['Price_Highs'] = group['High Price'] > group['High Price'].shift(long_window)
    group['Price_Lows'] = group['Low Price'] < group['Low Price'].rolling(window=long_window).min().shift(1)

    group['Volume_Highs'] = group['Volume'] > group['Volume'].shift(short_window)
    group['Volume_Lows'] = group['Volume'] < group['Volume'].rolling(window=short_window).min()

    group['Volume_MA_Short'] = group['Volume'].rolling(window=short_window).mean()  
    
    group['Volume Bullish Divergence'] = (group['Price_Lows']) & (group['Volume'] > group['Volume_MA_Short'])

    group['Volume Bearish Divergence'] = (group['Price_Highs']) & (group['Volume'] < group['Volume_MA_Short'])

    return group[['Volume Bullish Divergence', 'Volume Bearish Divergence']]

df[['Volume Bullish Divergence', 'Volume Bearish Divergence']] = df.groupby('Symbol', group_keys=False).apply(lambda group: volume_divergence(group))


C:\Users\user\AppData\Local\Temp\ipykernel_5664\52544657.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[['Volume Bullish Divergence', 'Volume Bearish Divergence']] = df.groupby('Symbol', group_keys=False).apply(lambda group: volume_divergence(group))


In [426]:
df[
    (df['Business Date']== datetime(2025,2,17).date()) & 
    (df['Volume Bearish Divergence']==True)
]

,Business Date,Symbol,Security Name,Open Price,High Price,Low Price,Close Price,Volume,Fifty Two Week High,Fifty Two Week Low,Market Capitalization,Date,200 Day EMA,50 Day EMA,MACD,RSI,15 Days High,30 Days High,60 Days High,120 Days High,200 Days High,15 Days Low,30 Days Low,60 Days Low,120 Days Low,200 Days Low,prev_15 Days High,prev_30 Days High,prev_60 Days High,prev_120 Days High,prev_200 Days High,prev_15 Days Low,prev_30 Days Low,prev_60 Days Low,prev_120 Days Low,prev_200 Days Low,Key Level,Last_7_Days_High,Last_7_Days_Low,7_days_Within_5_Percent,7_days_Bullish_Breakout,7_days_Bearish_Breakout,Last_15_Days_High,Last_15_Days_Low,15_days_Within_10_Percent,15_days_Bullish_Breakout,15_days_Bearish_Breakout,Last_30_Days_High,Last_30_Days_Low,30_days_Within_10_Percent,30_days_Bullish_Breakout,30_days_Bearish_Breakout,Last_45_Days_High,Last_45_Days_Low,45_days_Within_15_Percent,45_days_Bullish_Breakout,45_days_Bearish_Breakout,Breakout_Type,VSA Bullish,VSA Bearish,Volume Bullish Divergence,Volume Bearish Divergence
78885,2025-02-17,AHPC,Arun Valley Hydropower Development Co. Ltd.,265.0,269.7,264.5,265.0,100119.0,349.0,149.0,10197.21,2025-02-17,245.849828,262.299505,Bearish,50.871080,279.0,279.0,299.8,338.53,338.530,260.0,242.0,239.300,214.4670,144.530,267.2,281.9,308.2,338.53,338.530,242.0,239.30000,239.300,157.237,144.5300,252.0,279.0,260.0,False,False,False,279.0,260.0,True,False,False,279.0,242.0,False,False,False,281.9,239.30000,False,False,False,No Breakout,False,False,False,True
79119,2025-02-17,ANLB,Aatmanirbhar Laghubitta Bittiya Sanstha Limited,4946.9,5074.9,4870.0,5074.9,1659.0,6735.0,1777.6,3975.84,2025-02-17,3737.155653,4947.334210,Bearish,56.963344,5446.0,6735.0,6735.0,6735.00,6735.000,4765.0,4716.2,3901.625,2360.6975,1624.105,6735.0,6735.0,6735.0,6735.00,6735.000,4716.2,4073.21075,3555.195,2186.625,1591.6915,5000.0,5074.9,4800.0,False,False,False,5446.0,4765.0,False,False,False,6735.0,4716.2,False,False,False,6735.0,4073.21075,False,False,False,No Breakout,False,False,False,True
78970,2025-02-17,API,Api Power Company Ltd.,284.0,288.0,283.1,284.0,192106.0,390.0,162.0,17255.63,2025-02-17,251.130726,275.319278,Bullish,66.483516,293.0,293.0,313.0,370.50,370.500,269.0,253.1,246.200,233.7000,153.900,282.5,282.5,317.2,370.50,370.500,253.1,246.20000,246.200,168.150,153.9000,280.0,290.0,275.0,False,False,False,293.0,269.0,True,False,False,293.0,253.1,False,False,False,293.0,246.20000,False,False,False,No Breakout,False,False,False,True
78957,2025-02-17,BARUN,Barun Hydropower Co. Ltd.,488.0,493.5,476.5,478.5,113458.0,524.0,211.0,2563.87,2025-02-17,371.153123,445.539551,Bearish,55.244755,524.0,524.0,524.0,524.00,524.000,441.0,390.3,375.000,310.9000,211.000,458.0,458.0,472.0,472.00,472.000,390.3,375.00000,356.100,296.000,211.0000,422.0,508.7,469.5,False,False,False,524.0,441.0,False,False,False,524.0,390.3,False,False,False,524.0,375.00000,False,False,False,No Breakout,False,False,False,True
79131,2025-02-17,BGWT,Bhagawati Hydropower Development Company Limited,1137.0,1188.0,1137.0,1165.0,8448.0,1361.7,548.0,6931.14,2025-02-17,923.681380,1073.468016,Bearish,51.724138,1241.0,1241.0,1241.0,1241.00,1307.232,1060.0,935.3,920.000,857.6000,555.936,1133.0,1133.0,1133.0,1133.00,1307.232,935.3,920.00000,895.000,857.600,543.0720,990.0,1241.0,1060.0,False,False,False,1241.0,1060.0,False,False,False,1241.0,935.3,False,False,False,1241.0,920.00000,False,False,False,No Breakout,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79143,2025-02-17,ULHC,Upper Lohore Khola Hydropower Company Limited,632.0,638.0,605.1,620.1,54755.0,693.0,300.0,3161.29,2025-02-17,449.612385,547.948383,Bullish,62.017080,693.0,693.0,693.0,693.00,693.000,545.0,476.5,432.700,364.0000,317.100,586.5,586.5,586.5,586.50,586.500,476.5,455.00000,384.50

In [ ]:
# SUPPORT AND RESISTANCE 

""" The below code looks at the highest and lowest prices for each stock symbol, 
    comparing each price with the 10 prices before and after it to find high and lows 
    next step will find the point where the highest and and lowest points occured the 
    most and mark those as level .. the nearest point below the close price will be 
    marked as support and above as resistance """

df['Business Date'] = pd.to_datetime(df['Business Date'])
df = df.sort_values(by=["Symbol", "Business Date"])

def calculate_pivot_high_low(group, left=10, right=10):
    pivot_high = []
    pivot_low = []
    
    for i in range(len(group)):
        start = max(0, i - left)
        end = min(len(group), i + right + 1)
        
        window_high = group['High Price'].iloc[start:end]
        window_low = group['Low Price'].iloc[start:end]
        
        if group['High Price'].iloc[i] == window_high.max():
            pivot_high.append(group['High Price'].iloc[i])
        else:
            pivot_high.append(None)

        if group['Low Price'].iloc[i] == window_low.min():
            pivot_low.append(group['Low Price'].iloc[i])
        else:
            pivot_low.append(None)
    
    group['Pivot High'] = pivot_high
    group['Pivot Low'] = pivot_low
    return group

df = df.groupby('Symbol', group_keys=False).apply(calculate_pivot_high_low)



C:\Users\user\AppData\Local\Temp\ipykernel_5664\1768203249.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Symbol', group_keys=False).apply(calculate_pivot_high_low)


In [ ]:
#Smart money concept
""" Market Structure: The Breakout_Type column might show a Bullish Breakout (e.g., from the 7_days_Bullish_Breakout or 15_days_Bullish_Breakout),
    indicating a potential uptrend.
    Key Level: The stock is approaching a Key Level and shows signs of support (e.g., a bullish price action or divergence).
    Volume Divergence: The Volume Bullish Divergence could show that even though the price made lower lows,
    volume is increasing, indicating that smart money is accumulating.
    Volume Spread: The VSA Bullish column could confirm that there’s strong buying pressure in the market.

    Bearish Trade Setup:
    Market Structure: If the Breakout_Type shows a Bearish Breakout, it may indicate that smart money is starting to exit positions, 
    and a downtrend could follow.
    Key Level: A significant resistance level (Key Level) may hold, and price may fail to break above it, indicating distribution by smart money.
    Volume Divergence: Bearish Divergence occurs when the price is making higher highs but volume is decreasing,
    suggesting that the smart money is likely exiting the stock. """